We're gonna have some basic setup before we get to the focus.
Here we define the models and make a couple auxiliary functions:

In [16]:
import numpy as np
import matplotlib.pyplot as plt

# Defining linear and logistic models
def linear(x, slope, intercept):
    return slope*x + intercept

def logit(p):
    return np.log(p/(1-p))

def inv_logit(alpha):
    return np.exp(alpha)/(np.exp(alpha)+1)

def logistic(x, beta_0, beta_1):
    return inv_logit(linear(x, beta_0, beta_1))



In [17]:
# Calculate the gradient of vector feld f at v
def gradient(f, v):
    differential_length: float = 2**-16 # this can be adjusted

    n = len(v)
    grad = np.zeros(n)

    # I wonder if numpy has a better way to do this?
    for i in range(len(v)):
        d = np.zeros(n)
        d[i] = differential_length
        directional_derivative = (f(v+d/2) - f(v-d/2))/differential_length
        grad[i] = directional_derivative
    return grad

And here are parameters to decide what model we're using. **Edit this section** if you want to test with a different model.

In [18]:
# These can be changed to run different tests
# You need to change the loss function below also
model = linear
step_size = 2**-12
epoch_count = 2**14

# This gets the parameter count of the model
# (Which is one less than the number of arguments that model takes, since one of the arguments is the input vector)
import inspect
param_count = len(inspect.signature(model).parameters) - 1

In [19]:
# Defining the loss function
def squared_error(params, x, y):
    squared_error = (model(x, *params) - y)**2
    return squared_error

def log_loss(params, x, y):
    prob = model(x, *params)
    if y:
        return -np.log(prob)
    else:
        return -np.log(1-prob)

def mean_loss(pointwise_loss):
    def f(params, xdata, ydata, weights = None):
        n = len(xdata)
        if weights is None:
            weights = [1] * n
        losses = [pointwise_loss(params, xdata[i], ydata[i])*weights[i] for i in range(n)]
        return sum(losses)/n
    return f


loss = mean_loss(squared_error)

In [20]:

# These determine the training data
xdata = np.array([0, 1, 2])
ydata = np.array([0, 1.5, 2])

Here's the code for training the model using gradient descent:

In [21]:
def gradient_descent(params, xdata, ydata, weights, loss = loss):
    f = lambda theta: loss(theta, xdata, ydata, weights)
    for _ in range(epoch_count):
        grad = gradient(f, params)
        params -= grad*step_size
    return params

This is the response function.
The response function calculates the parameters which result from retraining with an additional datapoint *new* that is weighted by some amount *epsilon*.

In [22]:
# Influence functions are defined as the first-order Taylor approximation of this around epsilon = 0
# Or maybe the derivative if it's more convenient to subtract off the value at 0, I think I've seen both definitions
def response_function(new_data, epsilon, xdata, ydata):
    newxdata = np.concatenate((np.array([new_data[0]]), xdata))
    newydata = np.concatenate((np.array([new_data[1]]), ydata))
    weights = np.array([epsilon] + [1 for _ in xdata])
    params = gradient_descent(np.zeros(param_count), newxdata, newydata, weights)
    return params

Here's the influence function (on the parameters). It's the derivative of the response function.

In [23]:
def influence(z, epsilon):
    a = response_function(z, epsilon/2, xdata, ydata)
    b = response_function(z, -epsilon/2, xdata, ydata)
    return (a-b)/epsilon

This makes a plot. It will need considerable editing if you're doing a different test—I should probably set up something more versatile later.

In [25]:
epsilon = 0.001
delta = 1
z1 = (0,0)
z2 = (1,1.5)
z3 = (2,2)
y = response_function(z1, 0, xdata, ydata)
print(f'Initial model: {y}')
influence1 = influence(z1, epsilon)
influence2 = influence(z2, epsilon)
influence3 = influence(z3, epsilon)
linspace = np.linspace(xdata.min(), xdata.max(), 500)

x1 = y + influence1*delta
x2 = y + influence2*delta
x3 = y + influence3*delta

print(influence1)
print(influence2)
print(influence3)

plt.plot(linspace, model(linspace, *y), color='black',linewidth=2, label="Initial Model")
plt.plot(linspace, model(linspace, *x1), color='red',linewidth=1, label=f"$\hat\\theta + \mathcal{{I}}(0,0)$")
plt.plot(linspace, model(linspace, *x2), color='orange',linewidth=1, label=f"$\hat\\theta + \mathcal{{I}}(1,1.5)$")
plt.plot(linspace, model(linspace, *x3), color='blue',linewidth=1, label=f"$\hat\\theta + \mathcal{{I}}(2,2)$")
plt.scatter(xdata,ydata)
plt.legend()
plt.show()

TypeError: gradient_descent() missing 1 required positional argument: 'loss'

In [14]:
# Calculates the influence of z1 on the loss on z2
# This is a directional derivative of loss on z2 in the direction of the influence of z1 
def loss_influence(params, z1, z2, epsilon):
    point_loss = lambda theta: loss(theta, z2[0], z2[1])
    grad = gradient(point_loss, params)
    return np.dot(grad, influence(z1, epsilon))

print(loss_influence(y, (0,0), (1,0), epsilon))

data = [(0, 0), (1, 1.5), (2, 2)]

for i in range(3):
    for j in range(3):
        print(f'Cost Influence of {data[i]} on {data[j]}:')
        print(loss_influence(y, data[i], data[j], epsilon))

-0.13120544192640377
Cost Influence of (0, 0) on (0, 0):
-0.04716392861213299
Cost Influence of (0, 0) on (1, 1.5):
0.0374848913334374
Cost Influence of (0, 0) on (2, 2):
0.009495262163364715
Cost Influence of (1, 1.5) on (0, 0):
0.03702740217143974
Cost Influence of (1, 1.5) on (1, 1.5):
-0.07386963571736015
Cost Influence of (1, 1.5) on (2, 2):
0.03685298276811481
Cost Influence of (2, 2) on (0, 0):
0.009103179214047272
Cost Influence of (2, 2) on (1, 1.5):
0.03687138332158968
Cost Influence of (2, 2) on (2, 2):
-0.04580656857786415
